In [2]:
import re
import sys
import math
import time
import json
import csv
import pickle
# import tweepy
import random
import requests
import pprint
import numpy as np
import pandas as pd
import datetime as dt
from datetime import datetime
import scipy.stats as stat
from patsy import dmatrices
from langdetect import detect
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.dates as mdates
from scipy.stats import chisquare, ttest_ind
from matplotlib.dates import DateFormatter
from collections import defaultdict, Counter
from sklearn.preprocessing import MinMaxScaler
from matplotlib.ticker import FormatStrFormatter
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [3]:
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.max_columns', None)

In [4]:
data_root = '/home/haopeng/Data/Retraction/'
altmetric_start_date = datetime.strptime('2011-06-10', '%Y-%m-%d')
altmetric_end_date = datetime.strptime('2019-10-08', '%Y-%m-%d')
CIs = {'90': 1.645, '95': 1.96, '99': 2.576}
day = 24*3600

In [5]:
blog_df = pd.read_csv(data_root + 'blog_mentions_correction.csv', header = 0)
blog_set = set(blog_df.loc[~blog_df.maybe_news.isnull(), 'blog_name'])

def correct_blogs(data, error_blogs):
    for xid, paper in data.items():
        if 'blogs' in paper['posts']:
            blogs_li = paper['posts']['blogs']
            news_li = []
            if 'news' in paper['posts']:
                news_li = paper['posts']['news']
            blogs_li_new = []
            for item in blogs_li:
                try:
                    if item['author']['name'] in error_blogs:
                        news_li.append(item)
                    else:
                        blogs_li_new.append(item)
                except:
                    pass
            paper['posts']['news'] = news_li
            paper['posts']['blogs'] = blogs_li_new
    return data

In [6]:
outlets = pd.read_csv(data_root+'all_news_sources.csv', header=0)
top_outlets = set(outlets.loc[outlets.Include == 'y', 'Mention source'])

In [7]:
len(top_outlets)

27

### Load tweets

In [8]:
with open(data_root + 'tweets_processed.pickle', 'rb') as ofile:
    tid_date, tid_text, tid_author, tid_follow_cn, retweet_org_ids = pickle.load(ofile)

In [9]:
tid_date['1100001600121098241']

datetime.datetime(2019, 2, 25, 11, 56, 33)

In [10]:
# change tweet date format to str to keep it the same pattern as date on other platforms.
for tid, tdate in tid_date.items():
    tdate = tdate.strftime("%Y-%m-%dT%H:%M:%S")
    tid_date[tid] = tdate

In [12]:
len(tid_date)

159408

In [13]:
len(tid_text)

159408

In [14]:
len(retweet_org_ids)

87670

In [15]:
len(tid_follow_cn)

159408

### Load uncertainty tweets

In [639]:
all_tweets_df = pd.read_csv(data_root + 'revision/all_tweets_uncertain_label_revision_share.csv', quoting=csv.QUOTE_ALL, header=0, dtype={'tid': 'str'})


In [628]:
uncertain_tids = all_tweets_df.loc[all_tweets_df.majority_vote == 1, 'tid'].tolist()

In [629]:
len(uncertain_tids)

916

### Control papers

In [7]:
altid_data = {}

with open(data_root+"control_alt_papers.json", 'r') as ffile:
    for row in ffile:
        paper = json.loads(row)
        altid = str(paper['altmetric_id'])
        altid_data[altid] = paper
        
altid_data = correct_blogs(altid_data, blog_set)

In [8]:
len(altid_data)

31835

In [28]:
doi_control_aids ={}

with open(data_root+"doi_control_alt_ids.json", 'r') as ffile:
    for line in ffile:
        line = json.loads(line)
        doi_control_aids[line['doi']] = [str(x) for x in line['alt_ids']]

In [29]:
len(doi_control_aids)

6367

In [30]:
control_aid_doi = {}
for doi, ids in doi_control_aids.items():
    for alt_id in ids:
        control_aid_doi[alt_id] = doi

Correct blogs

In [3789]:
platform_stat = defaultdict(int)
# control papers can sometimes be mentioned by Retraction Watch (together with retracted papers)
# e.g., http://retractionwatch.com/2015/10/21/allegedly-junk-paper-published-by-bmc-journal/
blogs_men_cn = defaultdict(int)

for altid in altid_data:
    paper = altid_data[altid]
    for platform in paper['posts']:
        if paper['posts'][platform] != []:
            platform_stat[platform] += 1
        if platform == 'blogs':
            for post in paper['posts']['blogs']:
                try:
                    # blogs.add(post['author']['name'])
                    blogs_men_cn[post['author']['name']] += 1
                except:
                    pass

In [3790]:
# after blog correction
sorted(platform_stat.items(), key=lambda x: -x[1])

[('twitter', 14712),
 ('patent', 4259),
 ('facebook', 3603),
 ('wikipedia', 1992),
 ('blogs', 1977),
 ('news', 1886),
 ('policy', 1263),
 ('f1000', 931),
 ('googleplus', 643),
 ('video', 323),
 ('peer_reviews', 313),
 ('reddit', 265),
 ('q&a', 66),
 ('weibo', 34),
 ('misc', 32),
 ('pinterest', 12),
 ('linkedin', 4),
 ('book_reviews', 1)]

In [383]:
with open(data_root+'blog_mentions.csv', 'w') as f:
    f.write("blog_name,num_papers_mentioned\n")
    for name, cn in sorted(blogs_men_cn.items(), key=lambda x: x[1], reverse=True):
        f.write('"%s",%d\n'%(name, cn))

In [16]:
plat_cate = {
 'blogs': 'blog',
 'news': 'news_media',
 'patent': 'knowledge_repo',
 'f1000': 'knowledge_repo',
 'peer_reviews': 'knowledge_repo',
 'wikipedia': 'knowledge_repo',
 'q&a': 'knowledge_repo',
 'facebook': 'social_media',
 'googleplus': 'social_media',
 'linkedin': 'social_media',
 'pinterest': 'social_media',
 'reddit': 'social_media',
 'twitter': 'social_media',
 'video': 'social_media'
}

### Compare with retracted papers

In [18]:
doi_alt_data = {}

with open(data_root+"retraction_altmetric.json", 'r') as ffile:
    for row in ffile:
        record = json.loads(row)
        doi = record['citation']['doi'].lower()
        doi_alt_data[doi] = record
        
doi_alt_data = correct_blogs(doi_alt_data, blog_set)

In [19]:
with open(data_root+'df_retract_clean.pickle', 'rb') as file:
    retract_df = pickle.load(file)

In [20]:
retract_df.head(1)

,Record ID,Title,Subject,Institution,Journal,Publisher,Country,Author,URLS,ArticleType,RetractionDate,RetractionDOI,RetractionPubMedID,OriginalPaperDate,OriginalPaperDOI,OriginalPaperPubMedID,RetractionNature,Reason,Paywalled,Notes,altmetric_score,altmetric_pub_date,pub_date_corrected,usable_days,tweet_ids,tw_date_sort,fcn_date_sort,uncertainty,news_media,social_media,blog,knowledge_repo,top_news
0,23542,Hospital Admission and Readmission Among Homel...,(HSC) Medicine - Neurology;(SOC) Psychology;,Department of Neurology and Weill Institute fo...,Neurology,American Academy of Neurology,United States,Nicole Rosendale;Elan L Guterman;John P Betjem...,NaN,Research Article;,2020-06-04 00:00:00,10.1212/WNL.0000000000009565,32499376,2019-05-24 00:00:00,10.1212/wnl.0000000000007645,31127074,Retraction,+Concerns/Issues About Data;+Retract and Replace;,No,see: https://n.neurology.org/content/92/24/e2822,29.75,2019-05-24,2019-05-24 00:00:00,137,"[1133693569737187329, 1133763571790127106, 113...","[2019-05-29T11:16:24, 2019-05-29T15:54:34, 201...","[506, 273, 4836, 1845, 48163, 181, 935, 1322, ...",[],"[2019-05-28T15:33:34, 2019-05-28T18:38:39]","[2019-05-29T11:16:24, 2019-05-29T15:54:34, 201...",[],[],[]


In [21]:
len(retract_df)

6680

Focus on papers published after Alt db start date.

In [26]:
retract_df = retract_df.loc[retract_df.pub_date_corrected.apply(lambda dttt: (dttt-altmetric_start_date).total_seconds() > 0)]
retract_df.index = range(len(retract_df))

In [31]:
retract_df = retract_df.loc[retract_df.OriginalPaperDOI.isin(doi_control_aids)]
retract_df.index = range(len(retract_df))

In [51]:
len(retract_df)

3985

In [52]:
doi_ret_date = dict(zip(retract_df['OriginalPaperDOI'], retract_df['RetractionDate']))

Count total num of posts for 4k retracted papers

In [2995]:
cn = 0
for doi in doi_ret_date:
    paper = doi_alt_data[doi]
    if 'posts' in paper:
        for plat in paper['posts']:
            if plat in plat_cate:
                for item in paper['posts'][plat]:
                    cn += 1

In [2996]:
cn

95016

### Build retraction df for post retraction analysis (removing retraction-discussion posts)

(Start) Note: skip the below code block first; run it after the following two csv files have been created

In [841]:
posts_ = pd.read_csv(data_root+'label_after_ret_posts_notification.csv', header=0)

In [842]:
len(posts_)

735

In [843]:
exclude_obs = list(posts_.loc[posts_['is_retraction_related']==1, ['post_url', 'paper_doi']].to_records(index=False))
exclude_obs = [url + ' ' + doi for (url, doi) in exclude_obs]

In [844]:
len(exclude_obs)

546

In [845]:
tw_ = pd.read_csv(data_root+'label_after_ret_tweets.csv', header=0, dtype={'tid': str})

In [846]:
exclude_tids = tw_.loc[tw_.is_retraction_related==1, 'tid'].to_list()

In [847]:
len(exclude_tids)

2029

(End) Note: skip the above code block first.

In [2582]:
doi_alt_data_filter = {}

# in the first run, this dict stores after retraction posts that do not contain keyword "retract", which will be used for manual labelling.
plat_posts_filter = defaultdict(list)
validation = defaultdict(list)    

for doi, ret_date in zip(retract_df.OriginalPaperDOI, retract_df.RetractionDate):
    paper = doi_alt_data[doi]
    doi_alt_data_filter[doi] = paper.copy()
    if 'posts' in paper:
        temp = defaultdict(list)
        for plat in paper['posts']:
            if plat in plat_cate:
                for item in paper['posts'][plat]:
                    if plat == 'twitter':
                        tid = item['tweet_id']
                        if tid in tid_date:
                            tdate = tid_date[tid]
                            tdate = datetime.strptime(tdate, '%Y-%m-%dT%H:%M:%S')
                            tdelta = int((tdate-ret_date).total_seconds()/day)
                            text = tid_text[tid]
                            # for pre-retraction filtering (do it only for the 6-month before retraction period)
                            if tdelta < 0 and tdelta >= -180:
                                if 'retract' not in text.lower():
                                    temp[plat].append(item)
                            # for after-retraction filtering (keyword filtering + manual labelling)
                            # if the paper was retracted the end of day May 03, but its recorded date will be the beginning of the day.
                            if tdelta > 1 and tdelta <= 180:
                                if 'retract' in text.lower():
                                    item['text'] = tid_text[tid]
                                    item['posted_on'] = tdate
                                    validation[plat].append((item, doi))
                                # in the first run, use this code because `exclude_tids` has not been created :
                                # if 'retract' not in text.lower():
                                # in the second run, use this code:
                                if 'retract' not in text.lower() and tid not in exclude_tids:
                                    temp[plat].append(item)
                                    item['text'] = tid_text[tid]
                                    item['posted_on'] = tdate
                                    plat_posts_filter[plat].append((item, doi))
                    else:
                        if 'posted_on' in item:
                            tdate = item['posted_on'][:19]
                            tdate = datetime.strptime(tdate, '%Y-%m-%dT%H:%M:%S')
                            tdelta = int((tdate-ret_date).total_seconds()/day)
                            text = ''
                            if 'title' in item and item['title']:
                                text += item['title']
                            if 'summary' in item and item['summary']:
                                text = text + ' ' + item['summary']
                            # for pre-retraction filtering
                            if tdelta < 0 and tdelta >= -180:
                                try:
                                    if detect(text) == 'en' and 'retract' not in text.lower():
                                        temp[plat].append(item)
                                except:
                                    pass
                            # for after-retraction filtering (keyword filtering + manual labelling)
                            if tdelta > 0 and tdelta <= 180:
                                try:
                                    if detect(text) == 'en':
                                        if 'retract' not in text.lower():
                                            strr = item['url'] + ' ' + doi
                                            # use this condition only in the first run; remove it in the second run
                                            if strr not in exclude_obs:
                                                temp[plat].append(item)
                                                plat_posts_filter[plat].append((item, doi))
                                        else:
                                            validation[plat].append((item, doi))
                                except:
                                    pass
        doi_alt_data_filter[doi]['posts'] = temp

In [2122]:
# stores non-retraction-discussion posts
# before manual label: 3194
len(plat_posts_filter['twitter'])

1108

In [2123]:
plat_posts_filter['twitter'][:5]

[({'license': 'gnip',
   'citation_ids': [50548558],
   'author': {'tweeter_id': '28960735'},
   'tweet_id': '1178670811672125445',
   'text': '@_SharpTongue_ @conal_tuohy @IntheNow_tweet https://t.co/WGB9UIBhuc',
   'posted_on': datetime.datetime(2019, 9, 30, 13, 59, 54)},
  '10.1038/s41586-018-0651-8'),
 ({'license': 'gnip',
   'citation_ids': [50548558],
   'author': {'tweeter_id': '16399202'},
   'tweet_id': '1179412771215204354',
   'text': 'Quantification of ocean heat uptake from changes in atmospheric O2 and CO2 composition https://t.co/71IY8fe4LF',
   'posted_on': datetime.datetime(2019, 10, 2, 15, 8, 11)},
  '10.1038/s41586-018-0651-8'),
 ({'license': 'gnip',
   'citation_ids': [63095899],
   'author': {'tweeter_id': '415906140'},
   'tweet_id': '1172404474620473344',
   'text': 'Social Success of Perfumes. (arXiv:1909.05726v1 [physics.soc-ph]) https://t.co/qbSO5eJvvn',
   'posted_on': datetime.datetime(2019, 9, 13, 6, 59, 43)},
  '10.1371/journal.pone.0218664'),
 ({'license'

In [2124]:
# before manual label: 449
len(plat_posts_filter['news'])

124

In [2125]:
plat_posts_filter['news'][:1]

[({'title': 'People Should be Warned of the Adverse Health Effects of 5G Mobile Phone Microwave Radiation Technology - Global Research',
   'url': 'http://ct.moreover.com/?a=40165434741&p=1pl&v=1&x=NiLdmlSo7WkGjaKT-f1bAA',
   'license': 'public',
   'citation_ids': [60444118],
   'posted_on': '2019-09-11T12:59:31+00:00',
   'summary': 'Read the Irish Independent article here. *** Dear Adrian, I read with dismay your article in Saturdays Independent on the safety of 5G.',
   'author': {'name': 'Globalresearch',
    'url': 'http://www.globalresearch.ca/',
    'image': 'https://s3.amazonaws.com/weyland.altmetric.com/news_sources/logos/000/000/950/normal/Screen_Shot_2016-02-03_at_09.55.56.png?1454493377'}},
  '10.3390/magnetochemistry5020031')]

In [2126]:
# before manual label: 360
len(plat_posts_filter['blogs'])

41

In [2127]:
plat_posts_filter['blogs'][:1]

[({'title': 'Gravitational fields, fish excrement, and “putor” programs: How did this “pure, utter nonsense” get into in a peer-reviewed journal?',
   'url': 'https://retractionwatch.com/2019/07/23/gravitational-fields-fish-excrement-and-putor-programs-how-did-this-pure-utter-nonsense-get-into-in-a-peer-reviewed-journal/',
   'license': 'public',
   'citation_ids': [45426471],
   'posted_on': '2019-07-23T10:24:29+00:00',
   'summary': 'Sometimes, a paper comes along that is so revolutionary, it defies description. So rather than try to do justice to a recent paper in Parasitology Research, we’ll reproduce a few paragraphs here: A certain point between the center of the earth and the cent',
   'author': {'name': 'Retraction Watch',
    'url': 'http://www.retractionwatch.com/',
    'description': 'Tracking retractions as a window into the scientific process'}},
  '10.1007/s00436-018-6010-8')]

In [2093]:
# before manual label: 51
len(plat_posts_filter['wikipedia'])

3

In [2094]:
plat_posts_filter['wikipedia'][0]

({'title': 'Physician burnout',
  'url': 'http://en.wikipedia.org/?diff=prev&oldid=906540583#altmetric_citation_45dee31b-d556-40e4-bf16-e63de7a21225',
  'license': 'public',
  'citation_ids': [48595314, 48595314, 48595314],
  'posted_on': '2019-07-16T14:25:11+00:00',
  'summary': 'Physician burnout has been classified as a psychological syndrome that can be expressed as a prolonged response to due chronic occupational stressors.',
  'page_url': 'http://en.wikipedia.org/?curid=60275503',
  'wiki_lang': 'en',
  'author': {'name': 'JinkleMind',
   'url': 'http://en.wikipedia.org/wiki/User:JinkleMind'}},
 '10.1001/jama.2018.12615')

In [2095]:
# before manual label: 51
len(plat_posts_filter['q&a'])

0

In [2097]:
# before manual label: 51
len(plat_posts_filter['f1000'])

3

Manually label posts on three platforms

In [593]:
posts_ = []
for cate in ['news', 'blogs', 'wikipedia']:
    for item, doi in plat_posts_filter[cate]:
        url = item['url']
        if url:
            try:
                summ = item['summary']
            except:
                summ = ''
            try:
                title = item['title']
            except:
                title = ''
            posts_.append((title, summ, url, cate, doi_alt_data[doi]['citation']['title'], doi))

In [596]:
posts_ = pd.DataFrame(posts_, columns=['post_title', 'post_summmary', 'post_url', 'platform', 'paper_title', 'paper_doi'])
posts_.to_csv(data_root+'label_after_ret_posts_notification.csv', index=False, header=True)


In [791]:
posts_ = []
for item, doi in plat_posts_filter['twitter']:
    tid = item['tweet_id']
    dtt = item['posted_on']
    text = item['text']
    ret_dtt = doi_ret_date[doi]
    try:
        title = doi_alt_data[doi]['citation']['title']
    except:
        title = ''
    posts_.append((tid, dtt, text, title, ret_dtt, doi))

In [792]:
posts_ = pd.DataFrame(posts_, columns=['tid', 'posted_on', 'text', 'paper_title', 'retraction_date', 'paper_doi'])

In [793]:
posts_['is_retraction_related'] = ''

In [797]:
posts_.to_csv(data_root+'label_after_ret_tweets.csv', index=False, header=True)

Check that keyword filtering ("retract") indeed works

In [2584]:
len(validation['twitter'])

3137

In [2928]:
# all 50 tweets are about retraction.
random.sample(validation['twitter'], 50)

[({'license': 'gnip',
   'citation_ids': [4435747],
   'author': {'tweeter_id': '131644899'},
   'tweet_id': '662314825247301632',
   'text': 'Very noble of these authors to retract. We need more people willing to do things like this. https://t.co/FsH8jpuhfv',
   'posted_on': datetime.datetime(2015, 11, 5, 17, 5, 34)},
  '10.1177/0956797615597672'),
 ({'license': 'gnip',
   'citation_ids': [48937366],
   'author': {'tweeter_id': '18057073'},
   'tweet_id': '1061957821120552960',
   'text': 'RETRACTED - that study that showed Low Carb diets were associated with more heart disease - due to "concerns with data integrity and an undisclosed conflict of interest by the lead author" https://t.co/xZya3TJGzq',
   'posted_on': datetime.datetime(2018, 11, 12, 12, 24, 9)},
  '10.1002/clc.23047'),
 ({'license': 'gnip',
   'citation_ids': [16512392],
   'author': {'tweeter_id': '52790128'},
   'tweet_id': '914978953168867328',
   'text': "Wow, they retract a paper because the authors can't agree on 

In [2586]:
len(validation['blogs'])

900

In [2929]:
# all 50 blogs are about retraction.
random.sample(validation['blogs'], 50)

[({'title': 'Fluid mechanics article retracted with no explanation',
   'url': 'http://retractionwatch.com/2015/05/27/fluid-mechanics-article-retracted-with-no-explanation/',
   'license': 'public',
   'citation_ids': [4063536],
   'posted_on': '2015-05-27T13:30:00+00:00',
   'summary': 'An article published earlier this year has been retracted from the &nbsp;Journal of Heat Transfer. But the retraction notice gives no information about what was amiss. The article is entitled &ldquo;Neural Network Methodology for Modeling Heat Transfer in',
   'author': {'name': 'Retraction Watch',
    'url': 'http://www.retractionwatch.com/',
    'description': 'Tracking retractions as a window into the scientific process'}},
  '10.1115/1.4030039'),
 ({'title': 'Neuroscience paper retracted after colleagues object to data publication',
   'url': 'http://retractionwatch.com/2015/12/31/neuroscience-paper-retracted-after-colleagues-object-to-data-publication/',
   'license': 'public',
   'citation_ids': 

In [2587]:
len(validation['news'])

182

In [2930]:
# all 50 news are about retraction.
random.sample(validation['news'], 50)

[({'title': 'Study Claiming Video Games ‘Train’ Players for Real-World Violence Retracted',
   'url': 'http://ct.moreover.com/?a=29332441985&p=1pl&v=1&x=CdimKZIIbpdJY_vazzhshg',
   'license': 'public',
   'citation_ids': [15610323, 721141],
   'posted_on': '2017-01-23T18:58:13+00:00',
   'summary': 'SIGN UP FOR OUR NEWSLETTER Communication Research has officially retracted a report entitled “‘ Boom, headshot!?’: Effect of Video Game Play and Controller Type on Firing Aim and Accuracy ” by Jodi L.',
   'author': {'name': 'Breitbart News Network',
    'url': 'http://www.breitbart.com/',
    'image': 'https://s3.amazonaws.com/weyland.altmetric.com/news_sources/logos/000/001/439/normal/Screen_Shot_2016-01-06_at_12.20.18.png?1452082836'}},
  '10.1177/0093650212446622'),
 ({'title': '"Acid Bath Stem Cell" Breakthrough Debunked, Nature Retracts Papers',
   'url': 'http://www.ibtimes.com/acid-bath-stem-cell-breakthrough-debunked-nature-retracts-papers-1618604',
   'license': 'public',
   'cita

In [2932]:
# this one article is also about retraction.
validation['wikipedia']

[({'title': 'Retraction',
   'url': 'http://en.wikipedia.org/?diff=prev&oldid=710306701#altmetric_citation_1bbb8c68-a5ba-4219-add4-2b23e7c5a1a3',
   'license': 'public',
   'citation_ids': [4960221],
   'posted_on': '2016-03-16T04:55:35+00:00',
   'summary': 'A retraction is a public statement made about an earlier statement that withdraws, cancels, refutes, or reverses the original statement or ceases and desists from publishing the original statement.',
   'page_url': 'http://en.wikipedia.org/?curid=582691',
   'wiki_lang': 'en',
   'author': {'name': 'Sk741~enwiki',
    'url': 'http://en.wikipedia.org/wiki/User:Sk741~enwiki'}},
  '10.1371/journal.pone.0146193')]

In [2933]:
# all 50 fb posts are about retraction.
random.sample(validation['facebook'], 50)

[({'title': 'Dr.Oz-endorsed diet pill study was bogus, researchers admit',
   'url': 'https://www.facebook.com/permalink.php?story_fbid=10152404846385334&id=321154145333',
   'license': 'public',
   'citation_ids': [861358],
   'posted_on': '2014-10-22T17:10:03+00:00',
   'summary': 'Watch a paper be withdrawn! this paper on green bean coffee extract is being pulled by the authors as potentially fraudulent! http://www.cbsnews.com/news/dr-oz-endorsed-green-coffee-bean-diet-study-retracted/\nCan still get it on pubmed (for free).. http://',
   'author': {'name': 'begin2dig (b2d)',
    'url': 'https://www.facebook.com/321154145333',
    'facebook_wall_name': 'begin2dig (b2d)',
    'image': 'https://graph.facebook.com/321154145333/picture',
    'id_on_source': '321154145333'}},
  '10.2147/dmso.s27665'),
 ({'url': 'https://www.facebook.com/permalink.php?story_fbid=890931041079837&id=306574629515484',
   'license': 'public',
   'citation_ids': [20703315],
   'posted_on': '2018-04-02T19:13:35

In [2936]:
# all are about retraction.
validation['googleplus']

[({'title': 'RETRACTED: The impact of medically supervised injection centres on drug-related harms: a meta-analysis...',
   'url': 'https://plus.google.com/115794021272752859034/posts/BGLYUjbTEm2',
   'license': 'public',
   'citation_ids': [46099246, 46099246],
   'posted_on': '2018-09-27T09:47:15+00:00',
   'summary': 'RETRACTED: The impact of medically supervised injection centres on drug-related harms: a meta-analysis... RETRACTED: The impact of medically supervised injection centres on drug-related harms: a meta-analysisTom May, Trevor Bennett, Katy HollowayCentre for',
   'author': {'name': 'Barb Kueber',
    'url': 'https://plus.google.com/115794021272752859034',
    'image': 'https://lh3.googleusercontent.com/a-/AAuE7mB-wPFQwnYtnbE6YVV9LPzzXZxbqNWWiaKCP9duvA?sz=50',
    'id_on_source': '115794021272752859034'}},
  '10.1016/j.drugpo.2018.06.018'),
 ({'title': 'This article has been retracted at the request of the Editor-in-Chief and Authors, due to evidence of...',
   'url': 'ht

In [2937]:
# all are about retraction.
validation['reddit']

[({'title': "After a year of silence from the journal, Seralini's infamous 2012 paper has been officially retracted by Food and Chemical Toxicology",
   'url': 'http://www.reddit.com/r/skeptic/comments/1sa8mm/after_a_year_of_silence_from_the_journal/',
   'citation_ids': [954940],
   'posted_on': '2013-12-07T00:50:16+00:00',
   'author': {'name': 'firemylasers',
    'url': 'http://www.reddit.com/r/skeptic',
    'image': 'http://thumbs.reddit.com/t5_2qj8f.png?v=2e5665e72466598a00a5128ff06c52a4',
    'id_on_source': 'skeptic',
    'followers': 67028,
    'subreddit': 'skeptic'}},
  '10.1016/j.fct.2012.08.005'),
 ({'title': 'Ethical retraction',
   'url': 'http://www.reddit.com/r/Physics/comments/yh7rw/ethical_retraction/',
   'citation_ids': [895670],
   'posted_on': '2012-08-19T17:01:02+00:00',
   'author': {'name': 'iorgfeflkd',
    'url': 'http://www.reddit.com/r/physics',
    'image': 'http://thumbs.reddit.com/t5_2qhi6.png?v=473f549a8d28bd98af5407f6c865ef4b',
    'id_on_source': 'phy

In [2926]:
validation.keys()

dict_keys(['twitter', 'blogs', 'news', 'peer_reviews', 'facebook', 'googleplus', 'video', 'wikipedia', 'reddit'])

Create dataframe for retracted papers after filtering retraction-discussion posts

In [57]:
def get_time_series_list_filter(doi, category):
    # use filtered dict
    paper = doi_alt_data_filter[doi]
    time_li = []
    if 'posts' in paper:
        for plat in paper['posts']:
            # ignore misc.
            if plat in plat_cate:
                cate = plat_cate[plat]
                if cate == category:
                    for item in paper['posts'][plat]:
                        if plat == 'twitter':
                            tid = item['tweet_id']
                            if tid in tid_date:
                                time_li.append(tid_date[tid])
                        else:
                            time_li.append(item['posted_on'][:19])
    return sorted(time_li)

def get_top_news_time_series_list_filter(doi):
    paper = doi_alt_data_filter[doi]
    time_li = []
    if 'posts' in paper and 'news' in paper['posts']:
        for item in paper['posts']['news']:
            if 'posted_on' in item:
                tdate = item['posted_on'][:19]
                name = item['author']['name']
                # these top outlets are English language outlets, so we don't need to filter here.
                if name in top_outlets:
                    time_li.append(tdate)
    return sorted(time_li)

def get_tw_follow_sort_date_filter(doi):
    paper = doi_alt_data_filter[doi]
    lii = []
    if 'posts' in paper and 'twitter' in paper['posts']:
        for item in paper['posts']['twitter']:
            tid = item['tweet_id']
            if tid in tid_date:
                lii.append((tid_date[tid], tid_follow_cn[tid]))
    return sorted(lii, key=lambda x: x[0])

def get_uncertain_time_series_list_filter(doi):
    paper = doi_alt_data_filter[doi]
    time_li = []
    if 'posts' in paper and 'twitter' in paper['posts']:
        for item in paper['posts']['twitter']:
            tid = item['tweet_id']
            if tid in uncertain_tids: # non retweet
                if tid in tid_date:
                    time_li.append(tid_date[tid])
            else: # might be retweet or original tweet but is normal tweet
                if tid in retweet_org_ids:
                    org_id = retweet_org_ids[tid]
                    if org_id in uncertain_tids and tid in tid_date:
                        time_li.append(tid_date[tid])
    return sorted(time_li)

def get_tw_date_sort(lli):
    return [dtt for (dtt, fcn) in lli]

def get_tw_fcn_date_sort(lli):
    return [fcn for (dtt, fcn) in lli]

In [2128]:
retract_df_filter = retract_df.copy()

In [2129]:
retract_df_filter['news_media'] = retract_df_filter.OriginalPaperDOI.apply(get_time_series_list_filter, args=['news_media'])
retract_df_filter['social_media'] = retract_df_filter.OriginalPaperDOI.apply(get_time_series_list_filter, args=['social_media'])
retract_df_filter['blog'] = retract_df_filter.OriginalPaperDOI.apply(get_time_series_list_filter, args=['blog'])
retract_df_filter['knowledge_repo'] = retract_df_filter.OriginalPaperDOI.apply(get_time_series_list_filter, args=['knowledge_repo'])
retract_df_filter['top_news'] = retract_df_filter.OriginalPaperDOI.apply(get_top_news_time_series_list_filter)

In [2130]:
retract_df_filter['date_tweet_fcn'] = retract_df_filter.OriginalPaperDOI.apply(get_tw_follow_sort_date_filter)
retract_df_filter['tw_date_sort'] = retract_df_filter.date_tweet_fcn.apply(get_tw_date_sort)
retract_df_filter['fcn_date_sort'] = retract_df_filter.date_tweet_fcn.apply(get_tw_fcn_date_sort)
retract_df_filter = retract_df_filter.drop(columns=['date_tweet_fcn'])

In [2257]:
retract_df_filter['uncertainty'] = retract_df_filter.OriginalPaperDOI.apply(get_uncertain_time_series_list_filter)

In [2258]:
with open(data_root+'df_retract_clean_filter.pickle', 'wb') as file:
    pickle.dump(retract_df_filter, file)

In [2132]:
retract_df_filter.head(1)

,Record ID,Title,Subject,Institution,Journal,Publisher,Country,Author,URLS,ArticleType,RetractionDate,RetractionDOI,RetractionPubMedID,OriginalPaperDate,OriginalPaperDOI,OriginalPaperPubMedID,RetractionNature,Reason,Paywalled,Notes,altmetric_score,altmetric_pub_date,pub_date_corrected,usable_days,tweet_ids,tw_date_sort,fcn_date_sort,uncertainty,news_media,social_media,blog,knowledge_repo,top_news
0,23542,Hospital Admission and Readmission Among Homel...,(HSC) Medicine - Neurology;(SOC) Psychology;,Department of Neurology and Weill Institute fo...,Neurology,American Academy of Neurology,United States,Nicole Rosendale;Elan L Guterman;John P Betjem...,NaN,Research Article;,2020-06-04 00:00:00,10.1212/WNL.0000000000009565,32499376,2019-05-24 00:00:00,10.1212/wnl.0000000000007645,31127074,Retraction,+Concerns/Issues About Data;+Retract and Replace;,No,see: https://n.neurology.org/content/92/24/e2822,29.75,2019-05-24,2019-05-24 00:00:00,137,"[1133693569737187329, 1133763571790127106, 113...",[],[],[],[],[],[],[],[]


In [300]:
# with open(data_root+'df_retract_clean_filter.pickle', 'rb') as file:
#     retract_df_filter = pickle.load(file)